<a href="https://colab.research.google.com/github/bingxl/example/blob/master/python/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CNN 发展 Lenet(1998) -> AlexNet(2012) -> VGGNet(2014) -> InceptionNet(2014) -> ResNet(2015)

In [1]:
try:
    %tensorflow 2.x
except:
    pass

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Flatten, Dense, Dropout
print(f'tensorflow version: {tf.__version__}')

tensorflow version: 2.2.0-rc2


In [2]:
# 加载数据
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# 每个网络的输入层
input_layer = tf.keras.layers.InputLayer(input_shape=(32, 32, 3))

170500096/170498071 [==============================] - 6s 0us/step


In [0]:
def train(model, epochs=5, plot=True):
    model.compile(
        optimizer='adam',
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=['sparse_categorical_accuracy']
    )
    model.summary()
    history = model.fit(x_train, y_train, epochs=epochs, validation_data=(x_test, y_test))
    if plot:
        # draw loss and acc plot
        pass

In [6]:
def LeNet():
    # leNet最初始的卷积神经网络
    # LeNet还没有dropout， 激活函数常用sigmoid
    layers = tf.keras.layers
    model = tf.keras.Sequential([
        input_layer,
        layers.Conv2D(filters=6, strides=1, kernel_size=(5, 5), padding='valid', activation='sigmoid'),
        layers.MaxPool2D(pool_size=(2, 2), strides=2, padding='valid'),

        layers.Conv2D(filters=16, kernel_size=(5, 5), strides=1, padding='valid', activation='sigmoid'),
        layers.MaxPool2D(pool_size=(2, 2), strides=2, padding='valid'),

        layers.Flatten(),
        layers.Dense(120, activation='sigmoid'),
        layers.Dense(84, activation='sigmoid'),
        layers.Dense(10, activation='softmax')
    ])
    train(model, epochs=10)
    return model
LeNet()
    

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 28, 28, 6)         456       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 16)        2416      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 120)               48120     
_________________________________________________________________
dense_4 (Dense)              (None, 84)               

In [6]:
def AlexNet():
    # 使用relu激活函数提高准确率， 使用dropout缓解过拟合
    layers = tf.keras.layers
    model = tf.keras.Sequential([
        input_layer,

        layers.Conv2D(filters=96,kernel_size=3, strides=1, padding='valid'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPool2D(pool_size=(3, 3), strides=2, padding='valid'),

        layers.Conv2D(filters=256, kernel_size=(3, 3), strides=1, padding='valid'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPool2D(pool_size=(3, 3), strides=2),

        layers.Conv2D(filters=384, kernel_size=(3, 3), strides=1, padding='same', activation='relu'),

        layers.Conv2D(filters=384, kernel_size=(3, 3), strides=1, padding='same', activation='relu'),

        layers.Conv2D(filters=256, kernel_size=(3, 3), strides=1, padding='same', activation='relu'),
        layers.MaxPool2D(pool_size=(3, 3), strides=2),

        layers.Flatten(),
        layers.Dense(2048, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(2048, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ])
    train(model, epochs=10)
    return model

AlexNet()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 96)        2688      
_________________________________________________________________
batch_normalization (BatchNo (None, 30, 30, 96)        384       
_________________________________________________________________
activation (Activation)      (None, 30, 30, 96)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 12, 12, 256)       221440    
_________________________________________________________________
batch_normalization_1 (Batch (None, 12, 12, 256)       1024      
_________________________________________________________________
activation_1 (Activation)    (None, 12, 12, 256)       0

In [8]:
def VGG16Net():
    # 使用小尺寸卷积核，在减少参数的同时提高了识别准确率
    # 结构规整，非常适合硬件加速
    layers = tf.keras.layers
    
    model = tf.keras.Sequential([
        input_layer,

        # 1
        layers.Conv2D(filters=64, strides=1, kernel_size=3, padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),

        # 2
        layers.Conv2D(filters=64, strides=1, kernel_size=3, padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPool2D(pool_size=2, strides=2, padding='same'),
        layers.Dropout(0.2),

        # 3
        layers.Conv2D(filters=128, kernel_size=3, strides=1, padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),

        # 4
        layers.Conv2D(filters=128, strides=1, kernel_size=3, padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPool2D(pool_size=2, strides=2, padding='same'),
        layers.Dropout(0.2),

        # 5
        layers.Conv2D(filters=256, kernel_size=3, strides=1, padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),

        # 6
        layers.Conv2D(filters=256, kernel_size=3, strides=1, padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),

        # 7
        layers.Conv2D(filters=256, strides=1, kernel_size=3, padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPool2D(pool_size=2, strides=2, padding='same'),
        layers.Dropout(0.2),

        # 8
        layers.Conv2D(filters=512, kernel_size=3, strides=1, padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),

        # 9
        layers.Conv2D(filters=512, kernel_size=3, strides=1, padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),

        # 10
        layers.Conv2D(filters=512, strides=1, kernel_size=3, padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPool2D(pool_size=2, strides=2, padding='same'),
        layers.Dropout(0.2),

        # 11
        layers.Conv2D(filters=512, kernel_size=3, strides=1, padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),

        # 12
        layers.Conv2D(filters=512, kernel_size=3, strides=1, padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),

        # 13
        layers.Conv2D(filters=512, strides=1, kernel_size=3, padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPool2D(pool_size=2, strides=2, padding='same'),
        layers.Dropout(0.2),

        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(10, activation='softmax')
    ])
    train(model, epochs=10)
    return model
VGG16Net()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
batch_normalization_3 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
activation_3 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 64)       

In [15]:
# @TODO 未完成

layers = tf.keras.layers
class InceptionModel(tf.keras.Model):
    # 在同一层网络内使用不同尺寸的卷积核提升了模型感知力
    # 使用批标准化缓解了梯度消失
    def __init__(self, num_classes=10):
        super(InceptionModel, self).__init__()
        self.num_classes = num_classes

    def conv2d(self, ly, kernel_size, filters=16, strides=1, padding='same'):
        cv = layers.Conv2D(
            filters=filters,
            kernel_size=kernel_size,
            strides=strides,
            padding=padding
        )(ly)
        bn = layers.BatchNormalization()(cv)
        return layers.Activation('relu')(bn)

    def inceptionLayer(self, prevLayer, filters=16, strides=2):
        layers = tf.keras.layers
        l1 = self.conv2d(prevLayer, (1, 1), filters=filters, strides=strides)
        l21 = self.conv2d(prevLayer, (1,1), filters=filters, strides=strides)
        l2 = self.conv2d(l21, (3, 3), filters=filters, strides=strides)
        l31 = self.conv2d(prevLayer, (1,1), filters=filters, strides=strides)
        l3 = self.cov2d(l31, (5, 5), filters=filters, strides=strides)
        l41 = layers.MaxPool(pool_size=(3,3), strides=strides, padding='same')(prevLayer)
        l4 = self.conv2d(l41, (1,1), filters=filters, strides=strides)
        lout = layers.Concatenate()([l1, l2, l3, l4])
        return lout

    def call(self, x):
        filters = 16

        # 第一层
        ly = self.conv2d(input_layer(x), kernel_size = (3, 3), filters = filters)

        ly = self.inceptionLayer(ly, filters=filters, strides=2)
        ly = self.inceptionLayer(ly, filters=filters, strides=1)
        fiters *= 2
        ly = self.inceptionLayer(ly, filters=filters, strides=2)
        ly = self.inceptionLayer(ly, filters=filters, strides=1)
        ly = layers.GlobalAveragePooling2D()(ly)
        ly = layers.Dense(self.num_classes, activation='softmax')(ly)
        return ly


train(InceptionModel(), epochs=2)


ValueError: ignored